In [1]:
import selenium.webdriver as webdriver
from bs4 import BeautifulSoup
from pyvirtualdisplay import Display
from time import sleep
import datetime

import pandas as pd
from tqdm import tqdm

In [2]:
display = Display(visible=0,size=(1024, 768))
display.start()

In [3]:
driver = webdriver.Chrome('/home/ubuntu/chromedriver')

In [4]:
def get_table():
    
    def find_id(element):
        return str(element)[25:33]
    
    def chart(n):
        driver.get(f'https://www.genie.co.kr/chart/top200?pg={n}')
        page = driver.page_source
        soup = BeautifulSoup(page, 'lxml')
        
        table = soup.find(attrs={'class':'newest-list'}) \
                    .find_all('tr', attrs={'class':'list'})
        
        return [find_id(e) for e in table]
        
    return chart(1) + chart(2) + chart(3) + chart(4)

In [5]:
song_ids = get_table()

In [6]:
class song_page():
    
    def __init__(self, id):
        self.id = id
    
    def get_page(self):
        url = f"https://www.genie.co.kr/detail/songInfo?xgnm={self.id}"
        self.url = url

        driver.get(url)
        song_page = driver.page_source
        song_soup = BeautifulSoup(song_page, 'lxml')
        self.soup = song_soup
        
    def get_name(self):
        self.name = self.soup.find('h2',attrs={'class':'name'}).text.strip()

    def get_values(self):
        values = self.soup.find('ul',attrs={'class':'info-data'}) \
                          .find_all(attrs={'class':'value'})
        self.artist    = values[0].text
        self.album     = values[1].text
        self.genre     = values[2].text
        self.playtime  = values[3].text
        #self.lyricists = values[4].text
        #self.composers = values[5].text
        #self.arrangers = values[6].text

    def get_likes(self):
        self.likes = self.soup.find('em',attrs={'id':'emLikeCount'}).text

    def get_totals(self):
        counts = self.soup.find('div', attrs={'class':'total'}) \
                     .find_all('div')
        self.listners = counts[0].text.strip()
        self.numplays = counts[1].text.strip()
        
    def get_comments(self):
        self.comments = self.soup.find('span', attrs={'class':'article'}).text

    def get_now(self):
        self.now = datetime.datetime.now()

    def set_attributes(self):
        self.get_page()

        self.get_name()
        self.get_values()
        self.get_likes()
        self.get_totals()
        self.get_comments()
        self.get_now()

    def get_attributes(self):
        return (self.id, self.name,self.artist,self.album,self.genre,self.playtime,
                 #self.lyricists,self.composers,self.arrangers,
                 self.likes,
                 self.listners,self.numplays,
                 self.comments,
                 self.now)

### 곡 하나에 대해서 실험

In [7]:
first_song = song_page('90705547')
first_song.set_attributes()
first_song.get_attributes()

('90705547',
 'Dynamite',
 '방탄소년단',
 'Dynamite (DayTime Ver.)',
 '가요 / 댄스',
 '03:19',
 '89,099',
 '2,399,569',
 '108,797,742',
 '(총 2493개)',
 datetime.datetime(2021, 7, 1, 17, 28, 11, 157661))

### 모든 곡에 대해 적용

In [8]:
song_info = []

for sid in tqdm(song_ids):
    song = song_page(sid)
    song.set_attributes()
    song_info.append(song.get_attributes())

100%|██████████| 200/200 [02:54<00:00,  1.14it/s]


In [9]:
Songs = pd.DataFrame(song_info, 
                     columns = ['id',
                                'name','artist','album','genre','playtime',
                                #'lyricists','composer','arranger',
                                'likes',
                                'listners','numplays',
                                'comments',
                                'time(now)'])

In [10]:
Songs

,id,name,artist,album,genre,playtime,likes,listners,numplays,comments,time(now)
0,93619160,바라만 본다,MSG워너비 (M.O.M),MSG워너비 1집,가요 / 발라드,03:32,"15,345","806,630","6,355,651",(총 602개),2021-07-01 17:28:12.080672
1,93569588,비 오는 날 듣기 좋은 노래 (Feat. Colde),에픽하이 (EPIK HIGH),비 오는 날 듣기 좋은 노래,가요 / 랩/힙합,04:04,"5,955","372,880","1,105,145",(총 122개),2021-07-01 17:28:12.946962
2,93619161,나를 아는 사람,MSG워너비 (정상동기),MSG워너비 1집,가요 / 발라드,04:52,"9,969","679,318","3,169,729",(총 275개),2021-07-01 17:28:13.775589
3,93330349,Next Level,aespa,Next Level,가요 / 댄스,03:42,"23,575","1,160,117","18,786,805",(총 1094개),2021-07-01 17:28:14.657144
4,93300145,신호등,이무진,신호등,가요 / 락,03:52,"11,900","486,513","4,116,224",(총 100개),2021-07-01 17:28:15.600693
...,...,...,...,...,...,...,...,...,...,...,...
195,88729142,시든 꽃에 물을 주듯,HYNN (박혜원),시든 꽃에 물을 주듯,가요 / 발라드,03:19,"48,426","2,417,092","69,491,822",(총 229개),2021-07-01 17:31:02.435438
196,89275179,기억해줘요 내 모든 날과 그때를,거미 (Gummy),호텔 델루나 OST Part.7 (tvN 토일드라마),OST / 드라마,03:49,"41,225","2,537,546","62,838,081",(총 130개),2021-07-01 17:31:03.271917
197,87506735,All Falls Down (Feat. Juliander),Alan Walker & Noah Cyrus & Digital Farm Animals,Different World,POP / 일렉트로니카,03:19,"68,385","3,089,752","82,476,139",(총 149개),2021-07-01 17:31:04.194500
198,92345821,함께 했는데 이별은 나 혼자인 거야,소정 (레이디스 코드),함께 했는데 이별은 나 혼자인 거야,가요 / 발라드,03:25,"8,330","526,581","6,806,382",(총 25개),2021-07-01 17:31:05.027147


In [11]:
Songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         200 non-null    object        
 1   name       200 non-null    object        
 2   artist     200 non-null    object        
 3   album      200 non-null    object        
 4   genre      200 non-null    object        
 5   playtime   200 non-null    object        
 6   likes      200 non-null    object        
 7   listners   200 non-null    object        
 8   numplays   200 non-null    object        
 9   comments   200 non-null    object        
 10  time(now)  200 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(10)
memory usage: 17.3+ KB


In [13]:
#driver.quit()
#display.stop()